### Import modules

In [1]:
import keras_bitcoin
import dataset
import collections
import numpy as np

Using TensorFlow backend.


### Create Train/Test Dataset

In [3]:
df = dataset.get_labeled_dataset(number_of_file=12, from_date="2017-12-16")
print(collections.Counter(df["label"]))

Number of files loaded :  12
Counter({0: 9621, 1: 4981})


In [2]:
df = dataset.get_LDA_data()
df.head()

,Unnamed: 0,day,result,label
0,0,2015-12-15,http guy functioning government just post mod...,1
1,1,2015-12-16,say good goal fund just gain gaugeable post m...,0
2,2,2015-12-17,feature experience downvote filter effectiven...,1
3,3,2015-12-30,block run problem fudge doubt reddit subreddi...,0
4,4,2015-12-31,http sub ghost guy happy fund ad just genuine...,1


### Format Dataset

In [5]:
texts = df["text"]
labels = df["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size = keras_bitcoin.get_train_test_data(texts, labels)

In [3]:
texts = df["result"]
labels = df["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size = keras_bitcoin.get_train_test_data(texts, labels)

### Set Model Params

In [16]:
fun_dico = {
  "number_files_used": 12,
  "softplus_sigmoid": 0.7466990947723389,
  "softsign_sigmoid": 0.7289784550666809,
  "softsign_hard_sigmoid": 0.7338429689407349,
  "sigmoid_hard_sigmoid": 0.7515636086463928
}
epochs=20
batch_size=None
activations_functions=["sigmoid", "relu"]
dropouts = {}

### Train Mode

In [17]:
model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length, max_sentence_size, epochs=epochs, batch_size=batch_size, activations_functions=activations_functions, verbose=1)

C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
534/534 [==============================] - 0s 494us/step - loss: 0.7808 - acc: 0.4494
Epoch 2/20
534/534 [==============================] - 0s 148us/step - loss: 0.7320 - acc: 0.4494
Epoch 3/20
534/534 [==============================] - 0s 145us/step - loss: 0.6930 - acc: 0.4494
Epoch 4/20
534/534 [==============================] - 0s 127us/step - loss: 0.6603 - acc: 0.5599
Epoch 5/20
534/534 [==============================] - 0s 133us/step - loss: 0.6272 - acc: 0.8914
Epoch 6/20
534/534 [==============================] - 0s 129us/step - loss: 0.5946 - acc: 0.9382
Epoch 7/20
534/534 [==============================] - 0s 124us/step - loss: 0.5589 - acc: 0.9288
Epoch 8/20
534/534 [==============================] - 0s 120us/step - loss: 0.5193 - acc: 0.9345
Epoch 9/20
534/534 [==============================] - 0s 116us/step - loss: 0.4738 - acc: 0.9551
Epoch 10/20
534/534 [==============================] - 0s 110us/step - loss: 0.4254 - acc: 0.9738
Epoch 11/20
534/534 [========

###  Model Evaluation

In [18]:
loss, accuracy = model.evaluate(texts_test, labels_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

134/134 [==============================] - 0s 463us/step
Accuracy: 52.238804


###  Get Predictions

In [20]:
df_test = dataset.get_labeled_dataset(number_of_file = 10, from_date = "2017-02-10", date_included = False, all_files=True)
print(df_test.shape,df_test.head(2))

Number of files loaded :  668
(117152, 3)                                                 text  label        date
0                                               body      1  2015-12-15
1  Recent comment by him with regards to Gavin An...      1  2015-12-15


In [21]:
preds = keras_bitcoin.get_predictions(list(df_test["text"]), model, vocab_length, max_sentence_size)
df_test["preds"] = list(map(lambda x : int(x), preds))
df_test["correct"] = np.equal(preds, df_test["label"])

### Predictions Stats

In [22]:
dataset.get_prediction_stats(df_test)

Number Correct/Wrong Guess : 18267/98885
              Accuracy : 15.592563507238461
Invalid sentences count 82195


### Find best settings

In [23]:
def get_combinations(arr):
    combinations = []
    for i in range(len(arr)):
        item = arr[i]
        for j in range(i + 1, len(arr)):
            combinations.append([item, arr[j]])
    return combinations

In [26]:
import json
import pandas as pd

epochs=10
batch_size=None
perfs = {}
perfs["epochs"] = epochs
perfs["batch_size"] = batch_size
perfs["number_files_used"] = 12
dropout = {}

function_list = get_combinations(keras_bitcoin.available_activation_functions)
#function_list = keras_bitcoin.available_activation_functions
#function_list = [["softplus", "relu", "exponential", "softmax"]]
for function in function_list:
    print(function)
    
    if not isinstance(function, list):
        function = [function]
    else :
        function = list(function)
        #dropout[function[0]] = 0.1
        #dropout[function[2]] = 0.1
    model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length, max_sentence_size, epochs=epochs, batch_size=batch_size, activations_functions=function, verbose = 0)
    loss, accuracy = model.evaluate(texts_test, labels_test, verbose=0)
    perfs["_".join(function)] = [accuracy]
print(perfs)
#with open(f"./keras_stats/ep={epochs}_bats={batch_size}.json", "w") as fp:
#    json.dump(perfs, fp)

['tanh', 'elu']


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_77"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_77 (Embedding)     (None, 30, 20)            83800     
_________________________________________________________________
flatten_77 (Flatten)         (None, 600)               0         
_________________________________________________________________
dense_134 (Dense)            (None, 1)                 601       
_________________________________________________________________
dense_135 (Dense)            (None, 1)                 2         
Total params: 84,403
Trainable params: 84,403
Non-trainable params: 0
_________________________________________________________________
None
['tanh', 'softmax']


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_78 (Embedding)     (None, 30, 20)            83800     
_________________________________________________________________
flatten_78 (Flatten)         (None, 600)               0         
_________________________________________________________________
dense_136 (Dense)            (None, 1)                 601       
_________________________________________________________________
dense_137 (Dense)            (None, 1)                 2         
Total params: 84,403
Trainable params: 84,403
Non-trainable params: 0
_________________________________________________________________
None
['tanh', 'selu']


...

Model: "sequential_129"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_129 (Embedding)    (None, 30, 20)            83800     
_________________________________________________________________
flatten_129 (Flatten)        (None, 600)               0         
_________________________________________________________________
dense_238 (Dense)            (None, 1)                 601       
_________________________________________________________________
dense_239 (Dense)            (None, 1)                 2         
Total params: 84,403
Trainable params: 84,403
Non-trainable params: 0
_________________________________________________________________
None
['hard_sigmoid', 'linear']


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_130"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_130 (Embedding)    (None, 30, 20)            83800     
_________________________________________________________________
flatten_130 (Flatten)        (None, 600)               0         
_________________________________________________________________
dense_240 (Dense)            (None, 1)                 601       
_________________________________________________________________
dense_241 (Dense)            (None, 1)                 2         
Total params: 84,403
Trainable params: 84,403
Non-trainable params: 0
_________________________________________________________________
None
['exponential', 'linear']


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_131"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_131 (Embedding)    (None, 30, 20)            83800     
_________________________________________________________________
flatten_131 (Flatten)        (None, 600)               0         
_________________________________________________________________
dense_242 (Dense)            (None, 1)                 601       
_________________________________________________________________
dense_243 (Dense)            (None, 1)                 2         
Total params: 84,403
Trainable params: 84,403
Non-trainable params: 0
_________________________________________________________________
None
{'epochs': 10, 'batch_size': None, 'number_files_used': 12, 'tanh_elu': [0.46268656849861145], 'tanh_softmax': [0.5671641826629639], 'tanh_selu': [0.43283581733703613], 'tanh_softplus': [0.5671641826629639], 'tanh_softsign': [0.4328358173370

In [27]:
perfs

{'epochs': 10,
 'batch_size': None,
 'number_files_used': 12,
 'tanh_elu': [0.46268656849861145],
 'tanh_softmax': [0.5671641826629639],
 'tanh_selu': [0.43283581733703613],
 'tanh_softplus': [0.5671641826629639],
 'tanh_softsign': [0.43283581733703613],
 'tanh_relu': [0.43283581733703613],
 'tanh_sigmoid': [0.5373134613037109],
 'tanh_hard_sigmoid': [0.5746268630027771],
 'tanh_exponential': [0.5671641826629639],
 'tanh_linear': [0.43283581733703613],
 'elu_softmax': [0.5671641826629639],
 'elu_selu': [0.43283581733703613],
 'elu_softplus': [0.5597015023231506],
 'elu_softsign': [0.43283581733703613],
 'elu_relu': [0.4552238881587982],
 'elu_sigmoid': [0.4552238881587982],
 'elu_hard_sigmoid': [0.5522388219833374],
 'elu_exponential': [0.5671641826629639],
 'elu_linear': [0.43283581733703613],
 'softmax_selu': [0.0],
 'softmax_softplus': [0.0],
 'softmax_softsign': [0.43283581733703613],
 'softmax_relu': [0.43283581733703613],
 'softmax_sigmoid': [0.43283581733703613],
 'softmax_hard_

In [107]:
import os
import pandas as pd

directory = "./keras_stats/"
dicts = []
for file in os.listdir(directory):
    f = open(f'{directory}{file}')
    dicts.append(json.load(f))

functions = keras_bitcoin.available_activation_functions

perfs_stats = {"function" : [], "min" : [], "max" : [], "avg" : []}
for function in functions:
    sum_perfs = 0
    min_perfs = 1
    max_perfs = -1
    for dct in dicts:
        value = dct[function]
        sum_perfs += value
        if value > max_perfs:
            max_perfs = value
        if value < min_perfs:
            min_perfs = value
    perfs_stats["function"].append(function)
    perfs_stats["avg"].append(sum_perfs / len(functions))
    perfs_stats["max"].append(max_perfs)
    perfs_stats["min"].append(min_perfs)

print(pd.DataFrame(perfs_stats))
#softmax softplus softsign sigmoid hard_sigmoid  

        function       min       max       avg
0           tanh  0.000000  0.619942  0.382144
1            elu  0.000000  0.589487  0.054955
2        softmax  0.617438  0.617438  0.449046
3           selu  0.000000  0.622862  0.267266
4       softplus  0.415102  0.624948  0.409944
5       softsign  0.574885  0.617856  0.441006
6           relu  0.000000  0.622028  0.161262
7        sigmoid  0.600334  0.626617  0.445102
8   hard_sigmoid  0.592407  0.619942  0.445102
9    exponential  0.000000  0.624531  0.320514
10        linear  0.000000  0.618273  0.208367


In [59]:
functions = ["softmax", "softplus", "softsign", "sigmoid", "hard_sigmoid"]

In [14]:
import json

epochs = 5
batch_size = None
perfs = {}
perfs["number_files_used"] = 12
functions_list = [["sigmoid","hard_sigmoid"]]
dropout_value = 0.2
for functions in functions_list:
    functions = list(functions)
    dropouts = {}
    #dropouts[functions[0]] = 0.1
    #dropouts[functions[1]] = 0.08
    model = keras_bitcoin.get_model(texts_train = texts_train, labels_train = labels_train, vocab_length = vocab_length, max_sentence_size = max_sentence_size, epochs = epochs, 
    batch_size = batch_size, activations_functions = functions, verbose = 0, dropouts = dropouts)
    loss, accuracy = model.evaluate(texts_test, labels_test, verbose=0)
    #perfs["_".join(functions)] = accuracy
    print("acc : ", accuracy)


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1495, 20)          501200    
_________________________________________________________________
flatten_4 (Flatten)          (None, 29900)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 29901     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 2         
Total params: 531,103
Trainable params: 531,103
Non-trainable params: 0
_________________________________________________________________
None
acc :  0.6556636691093445


In [96]:
last_date = "2010-10-11"
for i in range(200):
    df_test = dataset.get_labeled_dataset(number_of_file = 5, from_date = last_date, date_included = False)
    preds = keras_bitcoin.get_predictions(list(df_test["text"]), model, vocab_length, max_sentence_size)
    df_test["preds"] = list(map(lambda x : int(x), preds))
    df_test["correct"] = np.equal(preds, df_test["label"])
    #dataset.get_prediction_stats(df_test)
    last_date = list(df_test["date"])[-1]

Number Correct/Wrong Guess : 2938/448
              Accuracy : 86.7690490253987
Invalid sentences count 0
Number Correct/Wrong Guess : 0/2560
              Accuracy : 0.0
Invalid sentences count 0
Number Correct/Wrong Guess : 3020/1403
              Accuracy : 68.27944833823197
Invalid sentences count 0
Number Correct/Wrong Guess : 1319/880
              Accuracy : 59.9818099135971
Invalid sentences count 0
Number Correct/Wrong Guess : 853/1376
              Accuracy : 38.26828174069089
Invalid sentences count 0
Number Correct/Wrong Guess : 1038/611
              Accuracy : 62.94724075197089
Invalid sentences count 0
Number Correct/Wrong Guess : 693/574
              Accuracy : 54.69613259668509
Invalid sentences count 0
Number Correct/Wrong Guess : 1418/332
              Accuracy : 81.02857142857142
Invalid sentences count 0
Number Correct/Wrong Guess : 702/1022
              Accuracy : 40.71925754060325
Invalid sentences count 0
Number Correct/Wrong Guess : 722/515
              Accu

ZeroDivisionError: division by zero

In [ ]:
 functions = ["softmax", "softplus", "softsign", "sigmoid", "hard_sigmoid"]
    ''' "softmax_softplus": 0.6580958962440491,
  "softmax_softsign": 0.6580958962440491,
  "softmax_sigmoid": 0.6580958962440491,
  "softmax_hard_sigmoid": 0.6580958962440491,
  "softplus_softsign": 0.6580958962440491,
  "softplus_sigmoid": 0.7466990947723389,
  "softplus_hard_sigmoid": 0.6580958962440491,
  "softsign_sigmoid": 0.7289784550666809,
  "softsign_hard_sigmoid": 0.7338429689407349,
  "sigmoid_hard_sigmoid": 0.7515636086463928'''
    
    {
  "softplus_sigmoid": 0.7466990947723389,
  "softsign_sigmoid": 0.7289784550666809,
  "softsign_hard_sigmoid": 0.7338429689407349,
  "sigmoid_hard_sigmoid": 0.7515636086463928
}
[["softplus", "sigmoid"], ["softsign", "sigmoid"], ["softsign", "hard_sigmoid"], ["sigmoid", "hard_sigmoid"]]